## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import confusion_matrix

/Users/aryehgelfand/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = pd.read_csv('chronic_kidney_disease_full.csv')

In [3]:
data.shape

(400, 25)

In [4]:
data.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [5]:
data.replace({'normal':1,'abnormal':0,'good':1,'poor':0,'yes':1,'no':0,'present':0,'notpresent':1,'ckd':1,'notckd':0},inplace=True)


In [6]:
def fill_na(data,col):
    dict_1 = dict(data[col].value_counts(normalize = True))# get normalized value counts in a dictionary
    keys = list(dict_1.keys())
    values = list(dict_1.values())
    for k,v in dict_1.items():
        data[col] = data[col].fillna(pd.Series(np.random.choice(keys,p=values, size=len(data))))

In [7]:
fill_na(data,'rbc')
fill_na(data,'rbcc')
fill_na(data,'wbcc')
fill_na(data,'pot')
fill_na(data,'sod')
fill_na(data,'pcv')
fill_na(data,'pc')
fill_na(data,'hemo')
fill_na(data,'su')

fill_na(data,'sg')
fill_na(data,'al')
fill_na(data,'bgr')
fill_na(data,'bu')
fill_na(data,'sc')
fill_na(data,'bp')
fill_na(data,'age')
fill_na(data,'ba')
fill_na(data,'pcc')
fill_na(data,'htn')
fill_na(data,'dm')
fill_na(data,'cad')
fill_na(data,'appet')
fill_na(data,'pe')
fill_na(data,'ane')



### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

- The relationship between blood pressure and CKD
- The relationship between hypertension and CKD
- Is there an association betwen diabetes and CKD

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [8]:
data.isnull().sum()

age      0
bp       0
sg       0
al       0
su       0
rbc      0
pc       0
pcc      0
ba       0
bgr      0
bu       0
sc       0
sod      0
pot      0
hemo     0
pcv      0
wbcc     0
rbcc     0
htn      0
dm       0
cad      0
appet    0
pe       0
ane      0
class    0
dtype: int64

In [9]:
data.describe()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.00000,400.000000,400.000000,400.000000,400.000000,400.00000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,51.507500,76.575000,1.017550,1.020000,0.445000,0.810000,0.762500,0.895000,0.945000,148.270000,58.080500,3.033375,136.92750,4.587000,12.570750,38.877500,8326.750000,4.71575,0.372500,0.345000,0.085000,0.795000,0.192500,0.152500,0.625000
std,17.023938,13.563898,0.005708,1.350615,1.093143,0.392792,0.426084,0.306937,0.228266,79.450728,51.176887,5.641981,11.81991,2.844064,2.853151,8.919439,2974.471068,1.02053,0.484076,0.475964,0.279231,0.404207,0.394757,0.359955,0.484729
min,2.000000,50.000000,1.005000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,1.500000,0.400000,4.50000,2.500000,3.100000,9.000000,2200.000000,2.10000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,42.000000,70.000000,1.013750,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,99.750000,27.000000,0.900000,135.00000,3.800000,10.400000,32.000000,6300.000000,3.97500,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,54.000000,80.000000,1.020000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,121.000000,42.000000,1.200000,138.00000,4.400000,12.700000,40.000000,7850.000000,4.80000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,64.000000,80.000000,1.020000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,162.250000,66.250000,2.800000,142.00000,4.900000,15.000000,45.000000,9800.000000,5.42500,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,90.000000,180.000000,1.025000,5.000000,5.000000,1.000000,1.000000,1.000000,1.000000,490.000000,391.000000,76.000000,163.00000,47.000000,17.800000,54.000000,26400.000000,8.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
def missing_values_table(df):
        mis_val = df.isnull().sum()# counts null values
        mis_val_percent = 100 * df.isnull().sum() / len(df)# get percentage null per colimn
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)# concats those columns together
        mis_val_table_ren_columns = mis_val_table.rename(# renames columns
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(# sorts by ascending
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns# prints datframe with columns and percentage missing
# took this function from stack overflow https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe
missing_values_table(data)

Your selected dataframe has 25 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

 - If we dropped any rows with missing values, we would have 400-152  = 248 left.
 - We don't have that many cases and we need every one, we cant lose 38% of our data.
 - The other columns have valuable data and we will lose it.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

 - The wrong data was inputed into the system
 - The Doctors and nurses collected the wrong information
 - Data was lost in the process of collection

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Tell everybody that they have CKD, you can never be wrong.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Specificity, False Negative

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

You will make people very scared unneccesarily. 

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Tell they eveyone they don't have CKD

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

We optimized for Sensitivy and minimized False Positives

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

A lot of people will not get the care they need.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [11]:
X = data.drop(['class'], axis = 'columns')
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.250,
                                                    random_state = 42,
                                                    stratify = y)


ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)



log_reg = LogisticRegression()

log_reg.fit(Z_train, y_train)
print(log_reg.score(Z_train, y_train))
print(log_reg.score(Z_test, y_test))


0.99
0.93


/Users/aryehgelfand/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [12]:
log_reg.coef_

array([[-0.16254301,  0.42930985, -1.4462779 ,  0.4870944 ,  0.34596863,
        -0.59759053, -0.63965727, -0.14237956, -0.05888164,  0.53262731,
         0.2003115 ,  0.62855534, -0.22438472, -0.67960167, -1.24656147,
        -1.29642214, -0.29683165, -0.55212325,  0.72478263,  0.78240458,
        -0.48881878, -1.18914547,  0.0086997 , -0.14149463]])

In [13]:
np.exp(-0.32304607/(1-0.32304607))

0.620515063277468

For every year lived a person is .62 times less likely to have CKD

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [14]:
np.exp(0.4727295/(1-0.4727295))

2.451155964351535

If a person has hypertension it makes them 2.45 times more likely to have CKD

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

It gives a binary outcome so it is easy to interpret.
It can handle a lot data types: integer, catgorical ordinal, and categorical with classes.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [15]:
preds = log_reg.predict(Z_test)
confusion_matrix(y_test, 
                 preds) 

# this code came from lecture 4.04 Mat Brems,David Yarrington

array([[37,  1],
       [ 6, 56]])

In [16]:
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()


In [17]:
pred_proba = [i[1] for i in log_reg.predict_proba(X_test)]

### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

We want to optimize for specificity, so that we won't have false negatives. 

In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,pred_proba)

0.5

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

yes, unbalanced classes will make the y-intercept inaccurate and skew the predictions accordingly. In this case, they are a problem.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

If we dont input the correct proportion of CKD to non CKD cases then the model will not be able to predict prevalence of the disease accurately. 
In order for the model to predict acccuratelly, it has to mimic the prevalence of the disease in the real world. 

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [27]:
from sklearn.utils import resample
np.random.seed(42)
df = pd.DataFrame()
CKD = data[data['class']==1]
boot = resample(CKD, replace=True, n_samples=200_000, random_state=42)
data = boot.append(C)

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [28]:
X = data.drop(['class'], axis = 'columns')
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.250,
                                                    random_state = 42,
                                                    stratify = y)

ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)
log_reg = LogisticRegression()
log_reg.fit(Z_train, y_train)
print(log_reg.score(Z_train, y_train))
print(log_reg.score(Z_test, y_test))

/Users/aryehgelfand/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


1.0
1.0


In [29]:
preds = log_reg.predict(Z_test)
confusion_matrix(y_test, 
                 preds) 

array([[18737,     0],
       [    0, 81363]])

Making a more accurate spread of the data made it significantly better at predicting values.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!

In [30]:
log_reg.coef_

array([[-0.79835082,  2.11305398, -7.23702359,  3.29637473,  2.59513381,
        -2.89953782, -0.90559149,  0.28403356, -1.19939323,  2.6828508 ,
         3.94992176, 14.68001748, -2.03468428, -6.91503038, -7.03664472,
        -3.77255338, -2.63003454, -3.84408201,  6.18684409,  5.76415746,
        -7.59095852, -6.25540457,  1.96463889, -1.09231538]])

In [31]:
data.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [33]:
np.exp(3.79850134/(1-3.79850134))

0.25734591788495553

If a person has hypertension it makes them .25 times more likely to have CKD